Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [178]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

In [179]:
State = namedtuple('State', ['taken', 'not_taken'])


def covered(state):
    '''
    this is a goal test, logical OR of corresponding elements of selected sets. if the reduced result is true, then the set is covered. it is a binary set covering!
    '''
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

#creates a tuple with name "State" and two fields of 'taken' and 'not_taken'; seems he has not used it and used a simple list for a state

# state definition: a set of items i take, a set of items i don't take.
# state = ({1,3,5},{0,2,4,6,7})   a possible state, taking second,fourth and sixth elements, not taking those other elements. 

#print(State(set(range(NUM_SETS)),set()))

In [180]:
# The goal is to solve the set-covering problem with the smallest number of tiles (smallest number of sets to cover)
PROBLEM_SIZE = 8
NUM_SETS = 20
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) 
for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

## Depth First

In [181]:
frontier = deque() #deque is a double-eneded queue. you can append and pop from both ends of the queue efficiently.
state = State(set(), set(range(NUM_SETS))) #the initial state, no set selected
frontier.append(state)
#print(frontier.)

counter = 0
current_state = frontier.pop() #extract one state(node) from frontier! this acts like FIFO (pop from right)
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:   #all the actions I can do in this state, (the paths we can take)
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            ) #this takes one of the sets in not_taken and put it into taken group, does this using  XOR "symmetric difference", outputs a number is not in both sets
            frontier.append(new_state)
        current_state = frontier.pop()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")
print(current_state.taken)
# Depth-First finds A solution very fast, but it is not optimal!


0it [00:00, ?it/s]

Solved in 12 steps (12 tiles)
{8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


In [182]:
current_state = State(set(), set(range(NUM_SETS))) #the initial state, no set 
print(state)
action = 11
new_state = State(current_state.taken ^ {action},                current_state.not_taken ^ {action})
print(new_state)

State(taken=set(), not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
State(taken={11}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19})


## Breadth First

In [183]:
frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.popleft() #pop from left, acts like LIFO
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.append(new_state)
        current_state = frontier.popleft()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")
print(current_state)
# the Breath-first result is the smallest number of tiles (each tree level is one more tile!) - so the Breath first solution is the optimal solution. but not efficient!

0it [00:00, ?it/s]

Solved in 1,905 steps (3 tiles)
State(taken={8, 12, 4}, not_taken={0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19})


## Greedy Best First

In [184]:
def f(state):
    '''this returns the number of items not covered as a measure of distance from the goal!'''
    missing_size = PROBLEM_SIZE - sum(covered(state))
    return missing_size

In [188]:
frontier = PriorityQueue()   # priority queue is sorted!
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state)) # pushing a tuple into the queue, the first one is used for sorting, here we give the distance to solution, so the min distance will pop first!

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")
print(current_state)
# not finding the optimal solution, but it is finding a (good) solution pretty fast

0it [00:00, ?it/s]

Solved in 13 steps (3 tiles)
State(taken={8, 4, 13}, not_taken={0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19})


## A*

In [186]:
def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def f(state):
    return len(state.taken) + h3(state)

In [187]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

0it [00:00, ?it/s]

Solved in 13 steps (3 tiles)
